# Madrid Pet Bussiness

Establish a dog kindergarten service in the city of Madrid

It is planned to examine the city of Madrid, in order to establish a dog kennel; So, you need to see in which parts of the city are places with greater and lesser concentration related to pets, whether they are nurseries, vets, veterinary emergencies, parks.

So we are going to cluster the places in the city and see where we can be located with the greatest probability of success.

Madrid was chosen for being the city with the most business possibilities, in terms of the size of the market in Spain, as well as being the city of Spain with the most canine pets, compared to other cities, both percentage-wise at the level of the general population. Madrid , as in number of canine pets.



## Why is this data useful for my business?

The general idea is to bring the relevant data to examine on the map of Madrid, which areas have a better environment to care for dogs.
Basically we will see that areas or neighborhoods of the city have more parks and services for pets.

The data of the neighborhoods and areas of Madrid were taken from the official data page of the Spanish government: https://datos.gob.es/es/catalogo/a13002908-barrios-del-munbierno-de-madrid

And the data of the places analyzed was taken from the foursquare.com API.



In [1]:
# import the libraries

import urllib.request
import pandas as pd
import numpy as np
from sklearn import preprocessing
import random
import requests
from pandas.io.json import json_normalize 
import json

### Get data about Madrid

In [ ]:
# Use geopy library to get the latitude and longitude values of Madrid City.

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: - 

In [ ]:
address = 'Madrid, ES'

geolocator = Nominatim(user_agent="md_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Madrid using latitude and longitude values
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=10)
   
map_madrid

### Go to Foursquare to get venues of Madrid 

In [ ]:
# The code was removed by Watson Studio for sharing.

### Let go to see parks of Madrid

In [ ]:
# Define the URL to get parks of Madrid.

# https://api.foursquare.com/v2/venues/search?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&query=QUERY&radius=RADIUS&limit=LIMIT

search_query = 'Parque'
radius = 10000
LIMIT = 3000

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
results = requests.get(url).json()
results

In [ ]:
parques = results['response']['venues']
    
parques = pd.json_normalize(parques) # flatten JSON

parques.head()

### Now we go to see relational bussiness

In [ ]:
# Define the URL to get other insteresting venues of Madrid.

# https://api.foursquare.com/v2/venues/search?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&query=QUERY&radius=RADIUS&limit=LIMIT

search_query = 'mascotas'
radius = 10000
LIMIT = 3000

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
results2 = requests.get(url2).json()
results2

In [ ]:
mascotas = results2['response']['venues']
    
mascotas = pd.json_normalize(mascotas) # flatten JSON

mascotas.head()

### Maybe it is better if we see them in a Madrid map

In [ ]:
lat = parques['location.lat']
lng = parques['location.lng']
lat2 = mascotas['location.lat']
lng2 = mascotas['location.lng']

# create map of Madrid using latitude and longitude values
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(parques['location.lat'], parques['location.lng'], parques['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    
for lat2, lng2, name in zip(mascotas['location.lat'], mascotas['location.lng'], mascotas['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat2, lng2],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc313e',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
     
map_madrid

### Where are we go to establish our bussiness? 

In [ ]:
# Just to look it

totaldf = parques.append(mascotas)
totaldf.shape


In [ ]:
# NaN values?

totaldf.columns[totaldf.isnull().any()]

In [ ]:
# Droping NaN values in my df

totaldf = totaldf.drop(['location.address', 'location.crossStreet', 'location.postalCode',
       'location.city', 'location.state', 'location.neighborhood',
       'venuePage.id'], axis = 1)

In [ ]:
# NaN values?

totaldf.columns[totaldf.isnull().any()]

In [ ]:
# let's see the columns of our df

totaldf.columns

In [ ]:
# Establish another df just to clustering

totaldfclust = totaldf.drop(['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.labeledLatLngs', 'location.distance', 'location.cc', 'location.country', 'location.formattedAddress'], axis = 1)

totaldfclust.head()

Clustering

Kmeans

In [ ]:
# Import some necesary libraries

import matplotlib.pyplot as plt # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

In [ ]:
# set number of clusters
kclusters = 5

# run k-means clustering
# kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(totaldfclust)

kmeans = KMeans(init = "k-means++", n_clusters = kclusters, n_init = 12)

kmeans.fit(totaldfclust)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster 

In [ ]:
 # add clustering labels
    
totaldfclust.insert(0, 'Km Clusters', kmeans.labels_)

totaldfclust.head()

Visualizing Kmeans clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(totaldfclust['location.lat'], totaldfclust['location.lng'], totaldfclust['Km Clusters']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# Our df 2.0

totaldfclust['name'] = totaldf['name']
totaldfclust.head()

finally go to see how is the best cluster to establish our bussiness 

In [ ]:
# Let's to grouping and counting the number of parks and pet's services in Madrid

groupfinal = totaldfclust.groupby('Km Clusters').count()
groupfinal

As we can see, cluster number 3 is our area of ​​the city of Madrid appropriate to establish our business because it has the largest number of businesses and places related to ours.